https://www.kaggle.com/yekenot/explore-ts-with-lstm

- inspired by 
     - https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/

- the author thinks a an MLP (normal NN) but then you remove the point of the time series 
     - https://machinelearningmastery.com/when-to-use-mlp-cnn-and-rnn-neural-networks/
     
     
This implementation uses the "classic" kera interface (it is still run by Tensorflow in the background). It could be rewritten to use keras within a more classic TF setup. See more for the difference between keras (with TF as runner) and keras inside TF: 

- https://www.pyimagesearch.com/2019/10/21/keras-vs-tf-keras-whats-the-difference-in-tensorflow-2-0/

In [4]:
import pandas as pd
import numpy as np
np.random.seed(10)

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense, LSTM

In [81]:
# load data
data = {
    'tra': pd.read_csv('./data/air_visit_data.csv'),
    'as': pd.read_csv('./data/air_store_info.csv'),
    'hs': pd.read_csv('./data/hpg_store_info.csv'),
    'ar': pd.read_csv('./data/air_reserve.csv'),
    'hr': pd.read_csv('./data/hpg_reserve.csv'),
    'id': pd.read_csv('./data/store_id_relation.csv'),
    'tes': pd.read_csv('./data/sample_submission.csv'),
    'hol': pd.read_csv('./data/date_info.csv').rename(columns={'calendar_date':'visit_date'})
    }

data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])

## data prep 
- and light feature engineering

In [88]:
# loop over air and hpg restaunrent (data specific)
for df in ['ar','hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date    
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    data[df] = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date'})

In [89]:
# Date formating... 
data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date
data['tra']['air_store_id'] = data['tra']['air_store_id'].map(lambda x: '_'.join(x.split('_')[:2]))


data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

In [90]:
# Groupby (store_ids features, day of week) --- (e.g. avg vistor etc)
unique_stores = data['tes']['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)

tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])

In [91]:
stores = pd.merge(stores, data['as'], how='left', on=['air_store_id']) 
lbl = LabelEncoder()
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date']) 
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date']) 

In [92]:
train = pd.merge(data['tra'], stores, how='left', on=['air_store_id','dow']) 
test = pd.merge(data['tes'], stores, how='left', on=['air_store_id','dow'])

for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])
    
train = train.fillna(-1)
test = test.fillna(-1)

In [93]:
display(train.head(2))
display(train.tail(3))

,air_store_id,visit_date,visitors,dow,year,month,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,7.0,23.843750,25.0,57.0,64.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,2.0,20.292308,21.0,54.0,65.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0


,air_store_id,visit_date,visitors,dow,year,month,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y
252105,air_24e8414b9b07decb,2017-04-20,7,3,2017,4,2.0,5.394737,6.0,8.0,38.0,11.0,72.0,35.653217,139.711036,-1.0,-1.0,-1.0,-1.0
252106,air_24e8414b9b07decb,2017-04-21,8,4,2017,4,2.0,5.738095,6.0,9.0,42.0,11.0,72.0,35.653217,139.711036,-1.0,-1.0,-1.0,-1.0
252107,air_24e8414b9b07decb,2017-04-22,5,5,2017,4,2.0,5.289474,6.0,8.0,38.0,11.0,72.0,35.653217,139.711036,-1.0,-1.0,-1.0,-1.0


# Loss function and train

In [95]:
def RMSLE(y, pred):
    return mean_squared_error(y, pred)**0.5

Part 1
Visitors as a feature to fit LSTM

Functions from https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras

## Normalize feature


In [96]:
train = train.sort_values('visit_date')
values = np.log1p(train['visitors'].values).reshape(-1,1)
values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [97]:
# Split into train and test sets

train_size = int(len(scaled) * 0.7)
test_size = len(scaled) - train_size

V_train, V_test = scaled[0:train_size,:], scaled[train_size:len(scaled),:]
print(len(V_train), len(V_test))


176475 75633


In [98]:
# Convert an array of values into a dataset matrix¶

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    print(len(dataY))
    return np.array(dataX), np.array(dataY)


In [99]:
# Create dataset with look back

look_back = 1
trainX, trainY = create_dataset(V_train, look_back)
testX, testY = create_dataset(V_test, look_back)


176474
75632


In [100]:
# Reshape X for model training¶

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [103]:
# Set up neural network

model = Sequential()
model.add(LSTM(4, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')


In [104]:
# Train LSTM with 3 epochs

history = model.fit(trainX, trainY, epochs=3, batch_size=100,
                    validation_data=(testX, testY), verbose=1,
                    shuffle=False) 


Epoch 1/3
1765/1765 [==============================] - 29s 3ms/step - loss: 0.0444 - val_loss: 0.0181
Epoch 2/3
1765/1765 [==============================] - 5s 3ms/step - loss: 0.0172 - val_loss: 0.0180
Epoch 3/3
1765/1765 [==============================] - 5s 3ms/step - loss: 0.0172 - val_loss: 0.0180


In [105]:
yhat = model.predict(testX)

yhat_inverse = scaler.inverse_transform(yhat.reshape(-1, 1))
testY_inverse = scaler.inverse_transform(testY.reshape(-1, 1))

RMSLE

In [106]:
rmsle = RMSLE(testY_inverse, yhat_inverse)
print('Test RMSLE: %.3f' % rmsle)


Test RMSLE: 0.816


# Part 2
Multivariate Forecast
Functions from https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras



In [108]:
train = train.sort_values('visit_date')
target_train = np.log1p(train['visitors'].values)

col = [c for c in train if c not in ['id', 'air_store_id', 'visitors']]

train = train[col]
train.set_index('visit_date', inplace=True)

train.head(2)

,dow,year,month,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y
visit_date,,,,,,,,,,,,,,,,
2016-01-01,4,2016,1,2.0,10.382353,10.0,20.0,68.0,2.0,3.0,33.581941,130.348436,-1.0,-1.0,-1.0,-1.0
2016-01-01,4,2016,1,3.0,18.656716,18.0,48.0,67.0,2.0,77.0,35.693840,139.703549,-1.0,-1.0,-1.0,-1.0


In [109]:
# Function to convert series to supervised learning

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # Forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg




In [110]:
# Normalize features

train['visitors'] = target_train
values = train.values
values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [113]:
# Frame as supervised learning

reframed = series_to_supervised(scaled, 1, 1)
reframed.head(2)

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),...,var8(t),var9(t),var10(t),var11(t),var12(t),var13(t),var14(t),var15(t),var16(t),var17(t)
1,0.666667,0.0,0.0,0.022556,0.076686,0.071895,0.023918,0.985714,0.214286,0.038835,...,0.971429,0.214286,0.757282,0.815045,0.968543,0.0,0.0,0.0,0.0,0.465644
2,0.666667,0.0,0.0,0.030075,0.132432,0.124183,0.055809,0.971429,0.214286,0.757282,...,1.000000,0.571429,0.650485,0.815973,0.968186,0.0,0.0,0.0,0.0,0.280179


In [114]:
reframed.drop(reframed.columns[[i for i in range(17,33)]], axis=1, inplace=True)
reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),var11(t-1),var12(t-1),var13(t-1),var14(t-1),var15(t-1),var16(t-1),var17(t-1),var17(t)
1,0.666667,0.0,0.0,0.022556,0.076686,0.071895,0.023918,0.985714,0.214286,0.038835,0.768135,0.904147,0.0,0.0,0.0,0.0,0.378435,0.465644
2,0.666667,0.0,0.0,0.030075,0.132432,0.124183,0.055809,0.971429,0.214286,0.757282,0.815045,0.968543,0.0,0.0,0.0,0.0,0.465644,0.280179
3,0.666667,0.0,0.0,0.082707,0.288725,0.274510,0.095672,1.000000,0.571429,0.650485,0.815973,0.968186,0.0,0.0,0.0,0.0,0.280179,0.205894
4,0.666667,0.0,0.0,0.015038,0.035856,0.026144,0.015945,0.857143,0.142857,0.902913,0.794438,0.940490,0.0,0.0,0.0,0.0,0.205894,0.351724
5,0.666667,0.0,0.0,0.120301,0.228969,0.209150,0.082005,1.000000,0.500000,0.300971,0.793805,0.938517,0.0,0.0,0.0,0.0,0.351724,0.150594


In [115]:
# Split into train and test sets¶

values = reframed.values
n_train_days = int(len(values) * 0.7)
train = values[:n_train_days, :]
test = values[n_train_days:, :]
# Split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# Reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(176474, 1, 17) (176474,) (75633, 1, 17) (75633,)


In [116]:
# Train LSTM with 3 epochs¶

multi_model = Sequential()
multi_model.add(LSTM(4, input_shape=(train_X.shape[1], train_X.shape[2])))
multi_model.add(Dense(1))
multi_model.compile(loss='mse', optimizer='adam')

In [117]:
multi_history = multi_model.fit(train_X, train_y, epochs=3,
                                batch_size=100, validation_data=(test_X, test_y),
                                verbose=1, shuffle=False)

Epoch 1/3
1765/1765 [==============================] - 8s 3ms/step - loss: 0.0237 - val_loss: 0.0179
Epoch 2/3
1765/1765 [==============================] - 5s 3ms/step - loss: 0.0169 - val_loss: 0.0176
Epoch 3/3
1765/1765 [==============================] - 5s 3ms/step - loss: 0.0169 - val_loss: 0.0177


In [119]:
# Make prediction

yhat = multi_model.predict(test_X)

In [121]:
# Apply invert scaling¶

test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# Invert scaling for forecast
inv_yhat = np.concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# Invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

In [122]:
#
rmsle = RMSLE(inv_y, inv_yhat)
print('Test RMSLE: %.3f' % rmsle)

Test RMSLE: 0.798
